In [2]:
pip install pyspark

     |████████████████████████████████| 204.2MB 63kB/s 
     |████████████████████████████████| 204kB 43.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=04201393bc92820600d6edc1c62403c2eaea0b61bb43fca3168345514335081b
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setAppName("practical session 1").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [ ]:
#part 1 lotr exercise
rdd_lotr = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_1/data/lotr.txt")

In [ ]:
rdd_lotr.count() #nof items in rdd

367

In [ ]:
rdd_lotr.first() #first item

'When Mr. Bilbo Baggins of Bag End announced that he would shortly be celebrating his eleventy-first birthday with a party of special magnificence, there was much talk and excitement in Hobbiton.'

In [ ]:
linesWithBilbo = rdd_lotr.filter(lambda line: "Bilbo" in line) #keeps lines with Bilbo in it

In [ ]:
linesWithBilbo.count()

71

In [ ]:
rdd_lotr.map(lambda line: len(line.split())).reduce(lambda a, b: a if (a > b) else b)

250

In [ ]:
test = rdd_lotr.map(lambda line: len(line.split()))

In [ ]:
test.take(5)

[31, 0, 154, 0, 18]

In [ ]:
test2 = rdd_lotr.filter(lambda x:x!="")

In [ ]:
test3= test2.map(lambda line: len(line.split()))

In [ ]:
test3.reduce(lambda a, b: a if (a > b) else b) #check for max 

250

In [ ]:
test3.reduce(min)

3

In [ ]:
wordCounts = rdd_lotr.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

In [ ]:
wordCounts.map(lambda x:(x[1],x[0])).sortByKey(False).take(5)

[(406, 'the'), (383, 'and'), (235, 'of'), (207, 'a'), (205, 'to')]

In [ ]:
linesWithBilbo.cache() #keep in cache memory if used repeatedly

PythonRDD[7] at RDD at PythonRDD.scala:53

In [ ]:
linesWithBilbo.count()

71

In [ ]:
#part 2 products
rdd_products = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_1/data/products.txt")

In [ ]:
rdd_products.take(5)

['1\tapple\t3\tfruit',
 '2\torange\t6\tfruit',
 '3\tbeer\t10\tbeverage',
 '4\twine\t15\tbeverage',
 '5\tTAOCP1\t20\tbook']

In [ ]:
rdd_products = rdd_products.map(lambda x:x.split()).map(lambda x:(int(x[0]),x[1],float(x[2]),x[3]))

In [ ]:
rdd_products.take(5)

[(1, 'apple', 3.0, 'fruit'),
 (2, 'orange', 6.0, 'fruit'),
 (3, 'beer', 10.0, 'beverage'),
 (4, 'wine', 15.0, 'beverage'),
 (5, 'TAOCP1', 20.0, 'book')]

In [ ]:
rdd_products.filter(lambda x:x[2]>20).map(lambda x:x[1])

['film1', 'film3', 'film4']

In [ ]:
new_rdd = rdd_products.map(lambda x:(x[3],x[2])).reduceByKey(max)

In [ ]:
new_rdd.take(10)

[('beverage', 15.0), ('book', 20.0), ('fruit', 7.0), ('film', 42.0)]

In [ ]:
rdd_modified = rdd_products

In [ ]:
rdd_modified.take(4)

[(1, 'apple', 3.0, 'fruit'),
 (2, 'orange', 6.0, 'fruit'),
 (3, 'beer', 10.0, 'beverage'),
 (4, 'wine', 15.0, 'beverage')]

In [ ]:
rdd_modified.map(lambda x:(x[3],1)).reduceByKey(lambda x,y: x+y).collect()

[('beverage', 4), ('book', 2), ('fruit', 4), ('film', 5)]

In [ ]:
rdd_modified.map(lambda x:(x[3],x[2])).groupByKey().map(lambda x:(x[0],sum(x[1])/len(x[1])))

[('beverage', 9.0), ('book', 12.5), ('fruit', 5.25), ('film', 27.6)]

In [ ]:
rdd_modified

In [5]:
#part 3 A, B

rdda = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_1/data/A.txt")
rddb = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_1/data/B.txt")

In [ ]:
rdda.collect()

['2 ', '4 ', '6 ', '8 ', '10', '10', '12', '14']

In [ ]:
rddb.collect()

['1 ', '3 ', '5 ', '7 ', '9 ', '10', '10', '10', '12', '14', '16', '18', '20']

In [ ]:
rdd_union = rdda.union(rddb).distinct()

In [ ]:
rdd_union.collect()

['10',
 '12',
 '1 ',
 '16',
 '20',
 '4 ',
 '6 ',
 '8 ',
 '5 ',
 '7 ',
 '18',
 '2 ',
 '14',
 '3 ',
 '9 ']

In [ ]:
rdd_intersection = rdda.intersection(rddb)

In [ ]:
rdd_intersection.collect()

['10', '12', '14']

In [ ]:
rdd_intersection.union(rddb.distinct()).collect()

['10',
 '12',
 '14',
 '1 ',
 '3 ',
 '10',
 '12',
 '14',
 '16',
 '20',
 '5 ',
 '7 ',
 '9 ',
 '18']

In [6]:
rddaa = rdda.map(lambda x:(x,1))
rddbb = rddb.map(lambda x:(x,1))

In [ ]:
rddaa.collect()

[('2 ', 1),
 ('4 ', 1),
 ('6 ', 1),
 ('8 ', 1),
 ('10', 1),
 ('10', 1),
 ('12', 1),
 ('14', 1)]

In [7]:
ra = rddaa.reduceByKey(lambda x,y:x+y)
rb = rddbb.reduceByKey(lambda x,y:x+y)

In [8]:
rab  = ra.union(rb).reduceByKey(min).map(lambda x:join(x))

In [9]:
def join(x):
  if x[1] == 1:
    return x[0]
  elif x[1] >1:
    return [x[0]]*x[1]

In [ ]:
rab.collect()

['2 ',
 ['10', '10'],
 '12',
 '14',
 '1 ',
 '3 ',
 '16',
 '20',
 '4 ',
 '6 ',
 '8 ',
 '5 ',
 '7 ',
 '9 ',
 '18']

In [10]:
#el
rddaa = rdda.map(lambda x:((x,"a"),1))
rddbb = rddb.map(lambda x:((x,"b"),1))

In [16]:
rddaa.collect()

[(('2 ', 'a'), 1),
 (('4 ', 'a'), 1),
 (('6 ', 'a'), 1),
 (('8 ', 'a'), 1),
 (('10', 'a'), 1),
 (('10', 'a'), 1),
 (('12', 'a'), 1),
 (('14', 'a'), 1)]

In [17]:
ra = rddaa.reduceByKey(lambda x,y:x+y)
rb = rddbb.reduceByKey(lambda x,y:x+y)

In [22]:
rel = ra.union(rb).map(lambda x:el(x)).reduceByKey(lambda x,y:(x[1],y[0]))

In [23]:
rel.collect()

[('1 ', (0, 1)),
 ('3 ', (0, 1)),
 ('10', (3, 2)),
 ('12', (1, 1)),
 ('14', (1, 1)),
 ('16', (0, 1)),
 ('20', (0, 1)),
 ('2 ', (1, 0)),
 ('4 ', (1, 0)),
 ('6 ', (1, 0)),
 ('8 ', (1, 0)),
 ('5 ', (0, 1)),
 ('7 ', (0, 1)),
 ('9 ', (0, 1)),
 ('18', (0, 1))]

In [20]:
def el(x):
  if x[0][1] == "a":
    return (x[0][0],(x[1],0))
  elif x[0][1] == "b":
    return (x[0][0],(0,x[1]))

In [24]:
def difference(x):
  if x[1][0]==0 or x[1][1] == 0:
    return x[0]
  else:
    return max(0,abs(x[1][0]-x[1][1]))*x[0]

In [25]:
rel.map(lambda x:difference(x)).filter(lambda x: len(x)>0).collect()

['1 ', '3 ', '10', '16', '20', '2 ', '4 ', '6 ', '8 ', '5 ', '7 ', '9 ', '18']

In [26]:
def symmetry(x):
  if x[1][0]==0 or x[1][1] == 0:
    return x[0]
  else:
    a = x[1][0]
    b = x[1][1]
    return (max(a,b) - min(a,b))*x[0]

In [27]:
rel.map(lambda x:symmetry(x)).filter(lambda x: len(x)>0).collect()

['1 ', '3 ', '10', '16', '20', '2 ', '4 ', '6 ', '8 ', '5 ', '7 ', '9 ', '18']